### Vorbereitung: [google-play-scraper](https://github.com/bolithium/google-play-scraper), [apkeep](https://github.com/EFForg/apkeep/) und [Android build-tools](https://developer.android.com/tools/releases/build-tools) (für dexdump) installieren.

In [ ]:
import sys
import os

# Colab-Beispielordner entfernen
!rm -r /content/sample_data

# Google Play Scraper
!git clone https://github.com/bolithium/google-play-scraper.git
os.makedirs('/content/apks', exist_ok=True)
module_path = '/content/google-play-scraper/'
if module_path not in sys.path:
    sys.path.append(module_path)

# APKeep
!wget -O apkeep https://github.com/EFForg/apkeep/releases/download/0.17.0/apkeep-x86_64-unknown-linux-gnu
!chmod +x apkeep

# Herunterladen der Android build-tools für dexdump, um classes*.dex-Dateien untersuchen zu können
#!sudo apt-get install openjdk-8-jdk
#!unzip tools.zip
#!rm tools.zip
!wget -O tools.zip https://dl.google.com/android/repository/build-tools_r34-rc3-linux.zip && unzip -o tools.zip && rm tools.zip

OLD_ENV_PATH = %env PATH
%env PATH=$OLD_ENV_PATH:/content/android-UpsideDownCake/

# temp-Verzeichnis für spätere Verwendung erstellen
os.makedirs('/content/temp/', exist_ok=True)

### App-Liste abfragen, DataFrame vorbereiten.

In [ ]:
# Top 100 aus allen Kategorien des Play Stores

from google_play_scraper import lists
import pandas as pd
# Gewählte Kategorie
collection = "topselling_free"
# Liste der Kategorien
category = [
    'ART_AND_DESIGN',
    'AUTO_AND_VEHICLES',
    'BEAUTY',
    'BOOKS_AND_REFERENCE',
    'BUSINESS',
    'COMICS',
    'COMMUNICATION',
    'DATING',
    'EDUCATION',
    'ENTERTAINMENT',
    'EVENTS',
    'FINANCE',
    'FOOD_AND_DRINK',
    'HEALTH_AND_FITNESS',
    'HOUSE_AND_HOME',
    'LIBRARIES_AND_DEMO',
    'LIFESTYLE',
    'MAPS_AND_NAVIGATION',
    'MEDICAL',
    'MUSIC_AND_AUDIO',
    'NEWS_AND_MAGAZINES',
    'PARENTING',
    'PERSONALIZATION',
    'PHOTOGRAPHY',
    'PRODUCTIVITY',
    'SHOPPING',
    'SOCIAL',
    'SPORTS',
    'TOOLS',
    'TRAVEL_AND_LOCAL',
    'VIDEO_PLAYERS',
    'WEATHER'
    ]

# Liste für alle Kategorien befüllen
temp_data = []
for cat in category:
  top_list = lists(cat, collection, num=100)
  temp_data += [{'appId': app['appId'], 'appName': app['title'], 'appCategory': cat} for app in top_list]

data = pd.DataFrame(temp_data)

display(data)

In [ ]:
# Exportiere die Top 100
data.to_csv('top100_all_category.csv')

### Apps herunterladen und XAPKs identifizieren.

In [ ]:
%%capture
# Apps mittels APKeep herunterladen
!mkdir apks
data.to_csv("/content/apks.csv", columns=['appId'], index=False, header=False)
!/content/apkeep -c /content/apks.csv -r 8 /content/apks/;
!rm /content/apks.csv

# Einige Apps werden als XAPK verteilt. Diese müssen anders behandelt werden und werden daher hier anhand der Dateiendung markiert.
data["isXAPK"] = data["appId"].apply(lambda id: os.path.isfile(f"/content/apks/{id}.xapk"))
# Die APKs mancher Apps sind nicht auf APK-Pure verfügbar. Falls diese nicht heruntergeladen werden konnten müssen diese später übersprungen werden.
data["apkDownloaded"] = data["appId"].apply(lambda id: os.path.isfile(f"/content/apks/{id}.xapk") or os.path.isfile(f"/content/apks/{id}.apk"))

In [ ]:
display(data)

### Definitionen zur Framework-Identifizierung


In [ ]:
frameworks = {
    'Flutter': {
        # Dateien, die sich direkt in der APK befinden. Angabe als Pfade, z.B. "res/layout/facebook_fragment_sso_login.xml"
        'files': ["lib/arm64-v8a/libflutter.so", "assets/flutter_assets/kernel_blob.bin"],
        # Strings, die in den classes*.dex-Dateien mittels dexdump gesucht werden. z.B. "com/facebook"
        'classes': ['io/flutter']
    },
    'React Native': {
        'files': ["libreactnativejni.so", "assets/index.android.bundle"],
        'classes': ['com/facebook/react']
    },
    'Cordova': {
        'files': ["assets/www/index.html", "assets/www/cordova.js", "assets/www/cordova_plugins.js", "org/apache/cordova"],
        'classes': []
    },
    'Xamarin': {
        'files': ["assemblies/assemblies.blob", "assemblies/assemblies.manifest"],
        'classes': ['com/xamarin']
    },
}

In [ ]:
# Untersuchung der APKs anhand verschiedener Merkmale, um das verwendete Framework zu bestimmen.
from zipfile import ZipFile
import re
import subprocess

# Erkennungsmerkmale Dateinamen
def detect_framework_via_files(apk: ZipFile):
    files_in_apk = apk.namelist()
    for framework in frameworks:
        indicator_paths = frameworks[framework]['files']
        if any(path in files_in_apk for path in indicator_paths):
            return framework

# Erkennungsmerkmale dexdump
def detect_framework_via_classes(appId, apk: ZipFile):
    for file in filter(lambda f: re.search("^classes\d*\.dex$", f), apk.namelist()):
      try:
        apk.extract(file, path=f"/content/temp/{appId}/")
        dexdump = subprocess.check_output(["dexdump", "-nje", f"/content/temp/{appId}/{file}"])

        for framework in frameworks:
            patterns = frameworks[framework]['classes']
            for pattern in patterns:
                grep = subprocess.Popen([f"grep", pattern], stdin=subprocess.PIPE, stdout=subprocess.PIPE)
                out, err = grep.communicate(input=dexdump)
                if len(out) > 0:
                  return framework
        os.unlink(f"/content/temp/{appId}/{file}")
      except:
        print("dexdump error")

def get_apk(appId, isXAPK):
    filepath = f"/content/apks/{appId}.{'x' if isXAPK else ''}apk"
    if not os.path.isfile(filepath):
        print(f"[{appId}] Datei existiert nicht: '{filepath}'")
        return None
    try:
        zip = ZipFile(filepath, 'r')
    except: # Falls der Download einer APK unterbrochen wird kann es sich um eine ungültige ZIP-Datei handeln
        return None
    # Falls es sich hierbei um eine XAPK handelt, befindet sich innerhalb der XAPK die gleichnamige APK, die eigentlich benötigt wird.
    if isXAPK:
        apkname = f"{appId}.apk"
        if apkname not in zip.namelist():
            return None
        # Die APK wird temporär extrahiert, um stattdessen in dieser nach den Framework-Dateien zu suchen
        zip.getinfo(apkname).filename = f"{appId}.apk"
        zip.extract(apkname, path="/content/apk/")
        zip.close()
        zip = ZipFile(f"/content/apk/{appId}.apk", 'r')
        # `zip` bezieht sich nun auf die APK innerhalb der XAPK

    return zip

def detect_single(entry):
    i, row = entry
    isXAPK = row["isXAPK"]

    zip = get_apk(row['appId'], isXAPK)
    if not zip:
        return

    # Zunächst Dateien in APK durchsuchen
    framework = detect_framework_via_files(zip)
    # Sonst Klassennamen durchsuchen
    if not framework:
      framework = detect_framework_via_classes(row['appId'], zip)
    # Falls immer noch nichts gefunden wurde ist es wahrscheinlich (?) Native
    if not framework:
        framework = "Unerkannt?"
    zip.close()
    if framework:
      print(f"detected fw: {framework} at id : {i} from app : {row['appId']}")
    data.at[i, "appFramework"] = framework

for i, row in data[data.apkDownloaded == True].iterrows():
  detect_single((i,row))

display(data)

# Ergebnis im CSV-Format abspeichern

data.to_csv('detected_frameworks.csv')

In [ ]:
display(data)

### Visualisierungen erstellen.

In [ ]:
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Anzahl der Frameworkerkennungen berechnen
framework_counts = df['appFramework'].value_counts()

# Anzeige in einem Bar-Chart
plt.figure(figsize=(8, 6))

# Daten einfügen
framework_counts.plot(kind='bar', color='skyblue')

# Titel und Beschriftungen einfügen
plt.title('Occurrences of appFramework')
plt.xlabel('Framework (Count)')
plt.ylabel('Count')

# Modify x-axis labels to include framework names and their counts
# Markierungen entlang der X-Achse setzen: <framework> (<count>)
plt.xticks(ticks=range(len(framework_counts)),
           labels=[f'{framework} ({count})' for framework, count in framework_counts.items()],
           rotation=45)

# Diagramm anzeigen
plt.tight_layout()
plt.show()

In [ ]:
len_dataset_downloaded = len(data[data.apkDownloaded == True])
print(len_dataset_downloaded)

len_gesamt_cp = data.appFramework.isnull().sum()
print(len_gesamt_cp)

anteil_flutter = len(data[data.appFramework == 'Flutter']) / len_dataset_downloaded
print(anteil_flutter)

anteil_react = len(data[data.appFramework == 'React Native']) / len_dataset_downloaded
print(anteil_react)

anteil_cordova = len(data[data.appFramework == 'Cordova']) / len_dataset_downloaded
print(anteil_cordova)

anteil_xamarin = len(data[data.appFramework == 'Xamarin']) / len_dataset_downloaded
print(anteil_xamarin)

sum_cp = anteil_flutter + anteil_react + anteil_cordova + anteil_xamarin
print(sum_cp)

print(data['appFramework'].value_counts())


In [ ]:
# Alle APK-Dateien die nicht Heruntergeladen wurden herausfiltern
result = data[data.apkDownloaded == True]

# Alle duplizierten Apps entfernen
result = result.drop_duplicates(subset=['appName'])

result.describe()

n = len(result)
print(f"Gesamtmenge : {n}")

cnt_ges = len(result[result.appFramework != 'Unerkannt?'])
ant_ges = round(((cnt_ges / n)*100), 2)
print(f"Davon CPF             : {cnt_ges}     in prozent: {ant_ges}%")


cnt_flutter = len(result[result.appFramework == 'Flutter'])
ant_flutter = round(((cnt_flutter / n)*100), 2)
print(f"Davon Flutter         : {cnt_flutter}     in prozent: {ant_flutter}%")

cnt_react = len(result[result.appFramework == 'React Native'])
ant_react = round(((cnt_react / n)*100), 2)
print(f"Davon React Native    : {cnt_react}     in prozent: {ant_react}%")

cnt_cordova = len(result[result.appFramework == 'Cordova'])
ant_cordova = round(((cnt_cordova / n)*100), 2)
print(f"Davon Cordova         : {cnt_cordova}      in prozent: {ant_cordova}%")

cnt_xamarin = len(result[result.appFramework == 'Xamarin'])
ant_xamarin = round(((cnt_xamarin / n)*100), 2)
print(f"Davon Xamarin         : {cnt_xamarin}      in prozent: {ant_xamarin}%")

In [ ]:
pie_data = {
    'Flutter': cnt_flutter,
    'React Native': cnt_react,
    'Cordova': cnt_cordova,
    'Xamarin': cnt_xamarin
}
total = sum(pie_data.values())

labels = []
sizes = []
for name, value in pie_data.items():
    percentage = value / total
    labels.append(f'{name}\n{percentage*100:.2f}% ({value})')
    sizes.append(value)

plt.rc('figure', figsize=(9,8.27))

title = plt.title('Erkannte Cross-Plattform-Frameworks')
plt.gca().axis("equal")
pie = plt.pie(sizes, startangle=0, labels=labels, labeldistance=0.76, textprops={'fontsize': 14, 'ha': 'center', 'va': 'center', 'weight': 'bold'})
plt.savefig("detected_cpf.png")